In [3]:
import pandas as pd
import numpy as np
import os
import glob

# Setting the output folder name to save output file
output_folder = r"E:\Script\GBM_Basin_RCP_Projection\Output\Processed_Gridded_Data"

# Iterating through all the file in the folder
for xlsx_file in glob.glob(r'E:\Script\GBM_Basin_RCP_Projection\Input_Data\Gridded_Data\*.xlsx'):

    # Splitting the file path to crate the file path list
    file_path_list = xlsx_file.split(os.sep)

    # Checking if a directory with the file name exist or not
    # If it does not exit then creating a directory as file name
    if os.path.exists(os.path.join(output_folder, '{}'.format(file_path_list[-1][:-5]))):
        pass

    else:
        os.mkdir(os.path.join(output_folder, '{}'.format(file_path_list[-1][:-5])))

    # Setting the output folder path
    output_folder_path = os.path.join(output_folder, '{}'.format(file_path_list[-1][:-5]))

    # Creating an ExcelFile object by reading the file
    excel_file = pd.ExcelFile(xlsx_file)

    # Defining a dataframe to create combined dataframe of gridded monthly data
    m_df = pd.DataFrame()

    # Iterating through each sheet's name
    for sheet in excel_file.sheet_names:

        # Parsing the sheet and convert it to a Pandas dataframe
        df = excel_file.parse(sheet)

        # Defining an empty list to store the temperature data of each grid point
        data_list = []

        # Iterating through each
        for lat, i in zip(df['lat\lon'], range((len(df['lat\lon'])))):

            # Iterating through each longitude to create a dataframe of gridded temperature data
            for col in df.columns:
                if col != 'lat\lon':
                    # Storing lat, lon and temperature data to a variable and
                    # updating it in the defined temperature list
                    x = lat, col, df[col][i]
                    data_list.append(x)

        # Creating a dataframe consisting gridded temperature data from the temperature list
        t_df = pd.DataFrame(data_list, columns=['Latitude', 'Longitude', 'Temperature (°C)'])

        # Saving the monthly gridded temperature dataframe
        monthly_output_file_path = os.path.join(output_folder_path, '{}.xlsx'.format(sheet))
        t_df.to_excel(monthly_output_file_path, float_format='%4f', index=False)

        # Updating the monthly dataframe in the combined dataframe
        if m_df.size == 0:
            # Renaming the temperature column name
            t_df.rename(columns={"Temperature (°C)": sheet}, inplace=True)
            m_df = pd.concat([m_df, t_df])

        # Merging monthly dataframes by latitude and Longitude columns
        else:
            t_df.rename(columns={'Temperature (°C)': sheet}, inplace=True)
            m_df = pd.merge(m_df, t_df, how='left', left_on=['Latitude', 'Longitude'],
                            right_on=['Latitude', 'Longitude'])

    # Inserting a point id column at the start ('0'th index) in the merged dataframe
    m_df.insert(0, 'Point_ID', ['Point_' + str(x + 1) for x in m_df.index.values])

    # Saving the combined monthly gridded temperature dataframe
    combined_monthly_file_path = os.path.join(output_folder_path,
                                              '{}_combined.xls'.format(file_path_list[-1][:-5]))
    m_df.to_excel(combined_monthly_file_path, float_format='%4f', index=False)

    print(file_path_list[-1][:-5])


C:\Users\HP\anaconda3\envs\py377\lib\site-packages\ipykernel_launcher.py:77: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.


Base_Temperature
Base_Temperature_2
